In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import itertools
import sys

sys.path.append("..")

from utility_functions import (calculate_model_performance,
                               plot_ROC,
                               one_hot_encode,
                               split_data_as,
                               random_shuffle)


def get_shapes(any_):
    for array in any_:
        try:
            print(array.shape)
        except:
            print("NONE")
    print("\n")


# ============= ACTIVATION FUNCTIONS ===============#

def sigmoid(Z, prime=False):
    # np.
    if prime:
        return sigmoid(Z) * (1 - sigmoid(Z))
    return 1 / (1 + np.exp(-Z))


def linear(Z, prime=False):
    if prime:
        return np.ones_like(Z)
    return Z


def relu(Z, alpha=0.01, prime=False):
    if prime:
        Z_relu = np.ones_like(Z, dtype=np.float64)
        Z_relu[Z < 0] = alpha
        return Z_relu
    return np.where(Z < 0, alpha * Z, Z)


def tanh(Z, prime=False):
    # np.tanh() could be used directly to speed this up
    if prime:
        return 1 - np.power(tanh(Z), 2)
    return (2 / (1 + np.exp(-2 * Z))) - 1


def elu(Z, prime=False):
    # https://mlfromscratch.com/activation-functions-explained/#/
    alpha = 0.2
    if prime:
        return np.where(Z < 0, alpha * (np.exp(Z)), 1)
    return np.where(Z < 0, alpha * (np.exp(Z) - 1), Z)


def softmax(Z, prime=False):
    # https://deepnotes.io/softmax-crossentropy
    # max(Z) term is added to stabilise the function.
    exps = np.exp(Z - np.max(Z))
    return exps / np.sum(exps, axis=0)


# ============== LOSS FUNCTIONS ===============#

# https://deepnotes.io/softmax-crossentropy
EPSILON = 1e-8


def calculate_error(Y, Y_hat):
    # Y and Y_hat should be in the form of (no_of_classes, no_of_training_examples)
    m = Y.shape[1]
    return -np.sum(Y * np.log(Y_hat + EPSILON)) / m


# References
# https://mc.ai/multilayered-neural-network-from-scratch-using-python/
# https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/
# https://www.coursera.org/learn/machine-learning/home/week/5
# https://www.coursera.org/specializations/deep-learning
# https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/src/network.py
# https://github.com/JWarmenhoven/Coursera-Machine-Learning

class NeuralNetwork:

    def __init__(
            self,
            input_layer: tuple,
            hidden_layer: list,  # list of tuples
            output_layer: int,
            batch_size=16,
            alpha=1,
            epoch=500,
            random_state=42,
            verbose=True,
            metrics="accuracy"
    ):
        self.input_layer = input_layer
        self.hidden_layer = hidden_layer
        self.output_layer = output_layer
        self.mini_batch_size = batch_size
        self.alpha = alpha
        self.epoch = epoch
        self.seed = random_state
        self.verbose = verbose
        self.metrics = metrics

        self.layers = len(self.weight_set_dimensions) + 1

    def get_A(self, X):
        A, _ = self.forwardpass(X)
        return A

    def get_Z(self, X):
        _, Z = self.forwardpass(X)
        return Z

    def display_information(self, X, Y, epoch_no):
        model_performance_metrics = calculate_model_performance(
            np.argmax(Y, axis=0),
            self.predict(X)
        )
        print("%s: %.10f - epoch %s    iteration %s - loss %.20f" % (
            self.metrics,
            model_performance_metrics[self.metrics],
            epoch_no,
            self.no_of_iterations,
            calculate_error(Y,
                            self.get_A(X)[-1])
        )
              )

    def get_dimensions_and_activations(self):
        self.dimensions = []
        self.activation_functions = []

        self.dimensions.append(self.input_layer[0])
        self.activation_functions.append(self.input_layer[1])

        for dim, act_func in self.hidden_layer:
            self.dimensions.append(dim)
            self.activation_functions.append(act_func)

        self.dimensions.append(self.output_layer)

    @property
    def weight_set_dimensions(self):
        self.get_dimensions_and_activations()
        a, b = itertools.tee(self.dimensions[::-1])
        next(b, None)
        weight_set_dimensions = list(zip(a, b))[::-1]
        return weight_set_dimensions

    def initialise_weights(self, layer=None):
        self.W = np.empty_like(range(self.layers), dtype=object)
        self.B = np.empty_like(range(self.layers), dtype=object)
        self.W[0] = None
        self.B[0] = None
        for layer, (y, x) in zip(range(1, self.layers), self.weight_set_dimensions):
            np.random.seed(self.seed)
            self.W[layer] = np.random.rand(y, x) / np.sqrt(self.dimensions[layer - 1])
            self.B[layer] = np.random.rand(y, 1)

    def forwardpass(self, X):
        Z = np.empty_like(range(self.layers), dtype=object)
        A = np.empty_like(range(self.layers), dtype=object)
        A[0] = X
        Z[0] = None
        for layer in range(1, self.layers):
            # activation_function starts from 0 whereas layer starts from 1
            active_function = self.activation_functions[layer - 1]
            arg_to_pass_to_eval = "(Z[layer])"

            Z[layer] = self.W[layer] @ A[layer - 1] + self.B[layer]
            A[layer] = eval(active_function + arg_to_pass_to_eval)
        return A, Z

    def backpropagation(self, Y, A, Z):
        self.delta = np.empty_like(range(self.layers), dtype=object)
        self.delta[0] = None

        self.gradient_W = np.empty_like(range(self.layers), dtype=object)
        self.gradient_B = np.empty_like(range(self.layers), dtype=object)
        self.gradient_W[0] = None
        self.gradient_B[0] = None

        self.delta[-1] = A[-1] - Y

        # We substract 1 here as delta_final is calculated seperately above
        for layer in reversed(range(1, self.layers - 1)):
            # 1 is substracted from layer as activations_functions start indexing from 0
            active_function = self.activation_functions[layer - 1]
            arg_to_pass_to_eval = "(Z[layer], prime=True)"


            self.delta[layer] = (
                    self.W[layer + 1].T @ self.delta[layer + 1] *
                    eval(active_function + arg_to_pass_to_eval)
            )

            # calculate the gradient

        for layer in range(1, self.layers):
            self.gradient_W[layer] = (self.delta[layer] @ A[layer - 1].T) / self.m
            self.gradient_B[layer] = np.sum(self.delta[layer], axis=1, keepdims=True) / self.m

        # update the weights
        for layer in range(1, self.layers):
            cost = calculate_error(Y, A[-1])
            cost_other = -np.mean(Y * np.log(A[-1] + 1e-8))
            self.W[layer] -= self.alpha * self.gradient_W[layer]
            self.B[layer] -= self.alpha * self.gradient_B[layer]


    def fit(self, X, Y):
        self.m = X.shape[1] # where (no_of_features, no_of_training_examples)
        self.initialise_weights()

        # By default the method is SGD(Stochastic Gradient Descent) if one wishes to use
        # the whole batch, simply pass the number of traning examples available as the
        # batch size when instantiating the class
        self.no_of_iterations = 0
        shuffled = np.arange(self.m)
        if self.verbose:
            print("Initialising weights...")
            print("Starting the training...")
            print("Initial cost: %.10f\n" % calculate_error(Y, self.get_A(X)[-1]))
        for epoch_no in range(1, self.epoch + 1):
            np.random.shuffle(shuffled)
            X_shuffled = X[:, shuffled]
            Y_shuffled = Y[:, shuffled]
            for i in range(0, self.m, self.mini_batch_size):
                self.no_of_iterations += 1
                X_mini_batch = X_shuffled[:, i: i + self.mini_batch_size]
                Y_mini_batch = Y_shuffled[:, i: i + self.mini_batch_size]

                A, Z = self.forwardpass(X_mini_batch)
                self.backpropagation(Y_mini_batch, A, Z)
                if self.no_of_iterations % 5000 == 0 and self.verbose:
                    self.display_information(X, Y, epoch_no)

    def predict(
            self,
            X: np.ndarray,
            return_prob_matrix=False
    ):
        """Predict the output given the training data.

            Returns the predicted values in two forms:

            1.either by picking up the highest value along the columns for every row,
                i.e. "np.argmax(self.A[-1].T, axis=1)"
            2.or by returning a matrix that is in the shape of Y.T where each column
                represents the probability of the instance belonging to that class.
                Please note that every column in Y.T represents a class. To be able to
                return the probability matrix, the final activation function must be
                softmax!
                i.e. "array([0.9650488423, 0.0354737543, 0.0005225966])"

        Args:
            X (numpy.ndarray): Training set in the shape of
                (no_of_features, no_of_training examples).
            return_prob_matrix (bool, optional): Returns the probability matrix if True.
                Defaults to False.

        Returns:
            numpy.ndarray:

            if return_prob_matrix is False, the output is in the shape of
                (no_of_training_examples, 1)
            if return_prob_matrix is True, the output is in the shape of
                (no_of_training_examples, no_of_features)
        """
        A, Z = self.forwardpass(X)
        if return_prob_matrix and self.activation_functions[-1] == "softmax":
            np.set_printoptions(precision=10, suppress=True)
            return A[-1].T
        return np.argmax(A[-1].T, axis=1)

# Testing with benchmark datasets

## 1.Iris Dataset

In [5]:
from sklearn.datasets import load_iris
data = load_iris()

x = data.data[:,[0,2]]
y = data.target

train, test = split_data_as(x, y, train=0.8, test=0.2)

X_train = train[:, :-1]
Y_train = train[:, -1].reshape(-1, 1)

Splitting the dataset as train and test...


In [6]:
print(X_train.shape)
print(Y_train.shape)
# print("\n")
# print(X_test.shape)
# print(Y_test.shape)
# print("\n")
# print(X_validation.shape)
# print(Y_validation.shape)

(120, 2)
(120, 1)


In [ ]:
model = NeuralNetwork(
    input_layer=(2, "relu"),
    hidden_layer=[(4, "relu"),
                  (4, "softmax")],
    output_layer=3,
    batch_size=16,
    alpha=2,
    epoch=250,
    random_state=42
)

In [10]:
from sklearn.grid_search import ParameterGrid

def grid_search(x, y, clf,**kwargs):

    grid = ParameterGrid(kwargs['param_grid'])

    models = {}
    for index, params in enumerate(grid):
        print(params)
        models[index] = clf(**params)
    return models

In [11]:
params = grid_search(
    X_train,
    Y_train,
    NeuralNetwork,
    param_grid={
        'batch_size': [8,16],
        'input_layer': [(2, 'relu')],
        'hidden_layer': [[(4,'relu'),(4,'softmax')], [(4,'sigmoid'),(4,'softmax')]],
        'output_layer': [3]
    }
)

{'output_layer': 3, 'batch_size': 8, 'hidden_layer': [(4, 'relu'), (4, 'softmax')], 'input_layer': (2, 'relu')}
{'output_layer': 3, 'batch_size': 8, 'hidden_layer': [(4, 'sigmoid'), (4, 'softmax')], 'input_layer': (2, 'relu')}
{'output_layer': 3, 'batch_size': 16, 'hidden_layer': [(4, 'relu'), (4, 'softmax')], 'input_layer': (2, 'relu')}
{'output_layer': 3, 'batch_size': 16, 'hidden_layer': [(4, 'sigmoid'), (4, 'softmax')], 'input_layer': (2, 'relu')}


In [ ]:
X_train.shape

In [7]:
from sklearn.grid_search import ParameterGrid

def grid_search(X, y, clf, n_fold=3, **kwargs):
    # X and y are in the shape of (no_of_training_examples, no_of_features)
    split_indices = np.int_(np.linspace(len(X)/n_fold, len(X), num=n_fold))
    dataset_shuffled = random_shuffle(X, y)

    splitted = np.array(
        np.split(
            dataset_shuffled,
            split_indices
        )[:-1]
    )

    models = {}
    results_dict = {}

    grid = ParameterGrid(kwargs['param_grid'])
    for i, params in enumerate(grid):
        models[i] = clf(**params)

        for index in range(n_fold):
            arrays_to_be_joined = np.delete(splitted, index, axis = 0)
            dataset_train = np.concatenate(arrays_to_be_joined)
            dataset_test = splitted[index]

            x_train = dataset_train[:, :-1].T
            y_train = one_hot_encode(dataset_train[:, -1]).T

            models[i].fit(x_train, y_train)

            x_test = dataset_test[:, :-1].T
            y_test = one_hot_encode(dataset_test[:, -1]).T

            results_dict["model_" + str(i + 1) + "_fold_" + str(index + 1)] = calculate_model_performance(
                np.argmax(y_test, axis=0),
                models[i].predict(x_test)
                )

    return results_dict, models

In [8]:
results_dict, models = grid_search(
    X_train,
    Y_train,
    clf=NeuralNetwork,
    n_fold=10,
    param_grid={
        'batch_size': [8,16],
        'input_layer': [(2, 'relu')],
        'hidden_layer': [[(4,'relu'),(4,'softmax')], [(4,'sigmoid'),(4,'softmax')]],
        'output_layer': [3],
        'verbose': [True]
    }
)

Initialising weights...
Starting the training...
Initial cost: 2.1230945565

accuracy: 92.5925925917 - epoch 358    iteration 5000 - loss 0.14965960157298166533
Initialising weights...
Starting the training...
Initial cost: 2.1278123396

accuracy: 92.5925925917 - epoch 358    iteration 5000 - loss 0.16098633642671719701
Initialising weights...
Starting the training...
Initial cost: 2.2074416857

accuracy: 92.5925925917 - epoch 358    iteration 5000 - loss 0.13339264529300626227
Initialising weights...
Starting the training...
Initial cost: 2.1065506638

accuracy: 95.3703703695 - epoch 358    iteration 5000 - loss 0.10605356950131443472
Initialising weights...
Starting the training...
Initial cost: 2.1060353884

accuracy: 93.5185185177 - epoch 358    iteration 5000 - loss 0.11509752915437758414
Initialising weights...
Starting the training...
Initial cost: 2.1626211316

accuracy: 95.3703703695 - epoch 358    iteration 5000 - loss 0.11797386107055972404
Initialising weights...
Starting t

In [12]:
results_dict

{'model_1_fold_1': {'F1': 99.9999999745,
  'accuracy': 99.99999999166667,
  'false_positive_rate': 0.0,
  'precision': 99.999999975,
  'prevalence': 33.333333330555554,
  'sensitivity/recall': 99.999999975,
  'specificity': 99.9999999875},
 'model_1_fold_10': {'F1': 99.99999996616667,
  'accuracy': 99.99999999166667,
  'false_positive_rate': 0.0,
  'precision': 99.99999996666666,
  'prevalence': 24.999999997916667,
  'sensitivity/recall': 99.99999996666666,
  'specificity': 99.99999998888889},
 'model_1_fold_2': {'F1': 90.9090908920661,
  'accuracy': 91.66666665902777,
  'false_positive_rate': 0.0,
  'precision': 99.99999998,
  'prevalence': 49.999999995833335,
  'sensitivity/recall': 83.33333331944445,
  'specificity': 99.99999998333334},
 'model_1_fold_3': {'F1': 79.99999996752,
  'accuracy': 91.66666665902777,
  'false_positive_rate': 0.0,
  'precision': 99.99999994999999,
  'prevalence': 24.999999997916667,
  'sensitivity/recall': 66.66666664444443,
  'specificity': 99.999999988888

In [ ]:
for value in results_dict.values():
    print(value["F1"])
    print("\n")

In [ ]:
Y_train.shape

In [ ]:
results_dict

In [ ]:
splitted[0]

In [ ]:
np.concatenate(np.delete(splitted, 1, axis = 0))

In [ ]:
splitted

In [ ]:
model.predict(X_train) == np.argmax(Y_train, axis=0)

In [ ]:
model_performance_metrics = calculate_model_performance(
    np.argmax(Y_train, axis=0),
    model.predict(X_train)
)

In [ ]:
model_performance_metrics

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
dt = data.data[:,[0,2]]
x_min, x_max = dt[:, 0].min() - 1, dt[:, 0].max() + 1
y_min, y_max = dt[:, 1].min() - 1, dt[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                     np.arange(y_min, y_max, 0.1))

# here "model" is your model's prediction (classification) function
Z = model.predict(np.c_[xx.ravel(), yy.ravel()].T) 

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(figsize=(12,8))
plt.contourf(xx, yy, Z,alpha=0.4)
#plt.axis('off')

plt.scatter(dt[:, 0], dt[:, 1], c=y,s=20, edgecolor='k')
plt.xlabel('sepal length')
plt.ylabel('petal length')

## 2.Make Moons dataset

In [ ]:
from sklearn.datasets import make_moons

x,y =make_moons(n_samples=1500, noise=.05)
X = x.T
Y = one_hot_encode(y).T

print(X.shape)
print(Y.shape)

In [ ]:
model = NeuralNetwork(
    input_layer=(2, "sigmoid"),
    hidden_layer=[(8, "tanh"),
                  (6, "relu"),
                  (4, "softmax")],
    output_layer=2,
    batch_size=64,
    alpha=0.5,
    epoch=2500,
    random_state=42
)


model.fit(X, Y)

In [ ]:
### Decision Boundaries
%matplotlib inline
import matplotlib.pyplot as plt
dt = x
x_min, x_max = dt[:, 0].min() - 0.5, dt[:, 0].max() + 0.5
y_min, y_max = dt[:, 1].min() - 0.5, dt[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                     np.arange(y_min, y_max, 0.1))

# here "model" is your model's prediction (classification) function
Z = model.predict(np.c_[xx.ravel(), yy.ravel()].T) 

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(figsize=(12,8))
plt.contourf(xx, yy, Z,alpha=0.4)
#plt.axis('off')
plt.scatter(dt[:, 0], dt[:, 1], c=y, s=20, edgecolor='k')
plt.title('Decision Boundaries')

## 3.Andrew NG Assignment 2 Dataset

In [ ]:
ex2data2 = np.loadtxt("../ex2/data/ex2data2.txt", delimiter=",")

X = ex2data2[:, :-1]
y = ex2data2[:, -1]

X = X.T
Y = one_hot_encode(y).T

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
model = NeuralNetwork(
    input_layer=(2, "relu"),
    hidden_layer=[(4, "relu"),
                  (4, "softmax")],
    output_layer=2,
    batch_size=64,
    alpha=0.5,
    epoch=25000,
    random_state=42
)

model.fit(X, Y)

In [ ]:
model_performance_metrics = calculate_model_performance(
    np.argmax(Y, axis=0),
    model.predict(X)
)

model_performance_metrics

In [ ]:
### Decision Boundaries
%matplotlib inline
import matplotlib.pyplot as plt
X = ex2data2

x1_min, x1_max = X[:, 0].min() - 0.3, X[:, 0].max() + 0.3,
x2_min, x2_max = X[:, 1].min() - 0.3, X[:, 1].max() + 0.3,
xx1, xx2 = np.meshgrid(np.linspace(x1_min, x1_max), np.linspace(x2_min, x2_max))

# here "model" is your model's prediction (classification) function
Z = model.predict(np.c_[xx1.ravel(), xx2.ravel()].T) 

negatives = ex2data2[ex2data2[:, -1] == 0]
positives = ex2data2[ex2data2[:, -1] == 1]

# Put the result into a color plot
Z = Z.reshape(xx1.shape)
plt.figure(figsize=(12,8))
plt.contourf(xx1, xx2, Z,alpha=0.4)
#plt.axis('off')
plt.scatter(negatives[:, 0], negatives[:, 1],s=50, color='k')
plt.scatter(positives[:, 0], positives[:, 1],s=50, color='r')
plt.title('Decision Boundaries')

plt.contour(xx1, xx2, Z, [0.5], linewidths=2, colors="g")

# MNIST dataset

In [ ]:
from scipy.io import loadmat
data = loadmat('../ex3/data/ex3data1.mat')
data.keys()

In [ ]:
x = data["X"]
y = data["y"]

In [ ]:
train, test, validation = split_data_as(x,y, train=0.6, test=0.2, validation=0.2)

X_train = train[:, :-1].T
Y_train = one_hot_encode(train[:, -1]).T

X_test = test[:, :-1].T
Y_test = one_hot_encode(test[:, -1]).T

X_validation = validation[:, :-1].T
Y_validation = one_hot_encode(validation[:, -1]).T

In [ ]:
print(X_train.shape)
print(Y_train.shape)
print("\n")
print(X_test.shape)
print(Y_test.shape)
print("\n")
print(X_validation.shape)
print(Y_validation.shape)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20,20))

sample = np.random.choice(data["X"].shape[0], 20)
ax.imshow(data["X"][sample,1:].reshape(-1,20).T)
ax.axis('off');

In [ ]:
model = NeuralNetwork(
    input_layer=(X_train.shape[0], "relu"),
    hidden_layer=[(10, "relu"),
                  (10, "softmax")],
    output_layer=Y_train.shape[0],
    batch_size=16,
    alpha=0.2,
    epoch=500,
    random_state=12
)


model.fit(X_train, Y_train)

In [ ]:
np.sum(model.predict(X_train) == np.argmax(Y_train,axis=0))

In [ ]:
calculate_model_performance(
    np.argmax(Y_train,axis=0),
    model.predict(X_train)
)

In [ ]:
for index, (act, predicted) in enumerate(zip(np.argmax(Y_train,axis=0), model.predict(X_train))):
    if act != predicted:
        fig, ax = plt.subplots(figsize = (2,2))
        ax.set_title("%s: act %s --- predicted %s" %(index, act + 1, predicted + 1))
        ax.imshow(X_train[:, index].reshape(-1,20).T)
        ax.axis('off');

In [ ]:
calculate_model_performance(
    np.argmax(Y_test,axis=0),
    model.predict(X_test)
)

In [ ]:
for index, (act, predicted) in enumerate(zip(np.argmax(Y_test,axis=0), model.predict(X_test))):
    if act != predicted:
        fig, ax = plt.subplots(figsize = (2,2))
        ax.set_title("%s: act %s --- predicted %s" %(index, act + 1, predicted + 1))
        ax.imshow(X_test[:, index].reshape(-1,20).T)
        ax.axis('off');

In [ ]:
def cross_validation():
    pass